# Step 1: Estimate a starting ρ from the length model by maximum likelihood.

The independent loss length model is a
Markov process known as an M/M/∞ queuing model
[28] (Figure 2A). In this queuing model, customers
(i.e., spacers) arrive according to a Poisson process with
rate λ. They are immediately served and exit after an
exponential waiting time with rate μ. The stationary distribution
of the number of busy servers (i.e., the number
of spacers in the array), is a Poisson distribution with
rate ρ:

ro = $$\rho = \frac{\lambda}{\mu}$$
with $\lambda$ = spacer insertion rate, $\mu$ = spacer_deletion_rate

prob_n_given_ro = $$p(n|\rho) = e^{-\rho} \frac{\rho^n}{n!}$$

## ML of $p(n|\rho)$ is the mean length of arrays

In [43]:
import numpy as np
from mpmath import *
mp.dps = 100; mp.pretty = True

arrays=[[9,2,3,4,5],[0,1,2,3,7,8],[1,10,11,12,13],[13,14]]
arrays=[[9,2,3,4,5],[0,1,2,3,7,8],[1,10,11,12,13]]


rho_init=mpf(sum([len(ar) for ar in arrays])/len([len(ar) for ar in arrays]))

rho_init

5.33333333333333303727386009995825588703155517578125

# Step 2: For each pair of spacers with overlap, generate the possible ancestors (doesnt need to actually generate the arrays)

Ancestral arrays can be arbitrarily
    large, but the probability of observing a certain
    length is given by p(n). For practical reasons we do
    not consider ancestors whose length is outside the
    central 99% of the stationary distribution given by ρ
    estimated in step 1, since they would have a
    negligible contribution to the likelihood. In detail, the
    length l1 where the cumulative distribution exceeds
    0.005 is the minimum ancestor length and the length
    l2 where the cumulative distribution exceeds 0.995 is
    the maximum ancestor length. Then the possible
    ancestor lengths n are between l1 and l2: l1 ≤ n ≤ l2.

### 2.1: Find pairs of spacers that overlap

In [44]:
from CRISPR_functions import is_overlapping
import itertools
from itertools import combinations

arrays=[[9,2,3,4,5],[0,1,2,3,7,8],[1,10,11,12,13],[14,15]]

overlapping_arrays=[pair for pair in list(itertools.combinations(arrays,2)) if is_overlapping(pair[0],pair[1])==1]

overlapping_arrays

[([9, 2, 3, 4, 5], [0, 1, 2, 3, 7, 8]),
 ([0, 1, 2, 3, 7, 8], [1, 10, 11, 12, 13])]

### 2.1: Find ancestor array size limits to exclude arrays outside of the 99% length distribution

In [45]:
from CRISPR_functions import get_limits_ancestor_sizes

size_lims=get_limits_ancestor_sizes(arrays)
size_lims

(0, 11)

### 2.2: Generate all possible ancestors for a pair within the size limits

In [46]:
from CRISPR_functions import CRISPR_pair
import importlib
importlib.reload(CRISPR_functions) 

pair=overlapping_arrays[0]
s1=pair[0]
s2=pair[1]
PAIR=CRISPR_pair(s1,s2)
print(pair)
print(PAIR.get_combi.__doc__)

# print(PAIR.get_combi(size_lims))

print('\n'.join([' : '.join([k,str(v)]) for k,v in PAIR.get_combi(size_lims).items()]))

([9, 2, 3, 4, 5], [0, 1, 2, 3, 7, 8])
 The function get_combi outputs a dictionary of all the possible combinations of spacers categories and their corresponding adjusted (per ancestor length) weights. 
        {c-i-j-u:ws} with:
        c number of spacers in common (spacers necessarily present in ancerstor), 
        i number of ancestral spacers amongst the spacers only present in array1, 
        j number of ancestral spacers amongt these only present in array2,
        u number of ancestral spacers lost in both array1 and array2,
        ws weight of each putative ancestral array from this combi.
        l1 (min ancestor length) and l2 (max ancestor length) have to be provided
        n length of ancestral array = sum(c,i,j,u) 
6-0-0-0 : 1.0
6-0-0-1 : 0.7777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777777778
6-0-1-0 : 0.1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
6-1-0-0 : 0.11111

# Step 3: For all pairs with overlap ...

## 3.1 Estimate the times with fixed ρ. 
It is possible to iterate through
the pairs and estimate their times
independently of the other pairs. The
estimation of both times is iterated
alternatingly until the likelihood has
converged.

In [47]:
import CRISPR_functions
from CRISPR_functions import OPTIMIZE_t1t2
import importlib
importlib.reload(CRISPR_functions) 
rho_init=mpf(sum([len(ar) for ar in arrays])/len([len(ar) for ar in arrays]))

print(OPTIMIZE_t1t2.__doc__)
t1t2_list=OPTIMIZE_t1t2(overlapping_arrays, rho_init, size_lims)
print(t1t2_list)

Provided a set of overlapping arrays and rho, OPTIMIZE_t1t2 finds their best respective divergence times t1,t2 from ancestor to arrays. The output is an array of [t1,t2] of length len(overlapping_arrays)
   direc: array([[1., 0.],
       [0., 1.]])
     fun: array(12.3604557)
 message: 'Optimization terminated successfully.'
    nfev: 44
     nit: 2
  status: 0
 success: True
       x: array([2.56219271, 2.5618453 ])
   direc: array([[1., 0.],
       [0., 1.]])
     fun: array(20.53228004)
 message: 'Optimization terminated successfully.'
    nfev: 46
     nit: 2
  status: 0
 success: True
       x: array([4.15804084, 4.17274828])
[(2.5621927111282803, 2.5618452989812335), (4.1580408364803265, 4.172748284661945)]


## 3.2 Estimate ρ with fixed times using L(ρ|t, S).

In [49]:
import CRISPR_functions
from CRISPR_functions import CRISPR_pair,OPTIMIZE_rho
import importlib
importlib.reload(CRISPR_functions) 

pair_list=[CRISPR_pair(pair[0],pair[1]) for pair in overlapping_arrays]
non_overlapping_arrays=[pair for pair in list(itertools.combinations(arrays,2)) if is_overlapping(pair[0],pair[1])==0]
print(size_lims)
OPTIMIZE_rho(t1t2_list,pair_list,size_lims,non_overlapping_arrays)

(0, 11)
69.04744453059309327310192030537502961657478536749045672869789368538947991918685466309794080385856112
69.04744453059309327310192030537502961657478536749045672869789368538947991918685466309794080385856112
55.27185130594058921621375092335297017425493758252110585805876275047294604339365227357067644194629688
48.43347147386464257803353985023716062373127840300154595625110249196055006531030055641637104839284388
48.74062378628979448785994836599986648282220662715861830143171294036802773013881980770371166825828805
52.92098470785755693151665007349689808591768016718402100019586849449382067990985408213937694367273493
48.43347147386464257803353985023716062373127840300154595625110249196055006531030055641637104839284388
48.70407296496342706033346434932549144346586886198279991788255152744621935413625949845547886587431469
49.66591262528164200888442079699184592794274024006369736517432390051980593237214832070502060316028885
48.25802250772672689694668303848682670553681876185151397349636410912315262

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

## 3.3 Check if the log-likelihood of the estimated parameters has converged,
then return the
estimated parameters, else repeat step 3.1
with the new parameters.